# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [108]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.sql.functions import udf, concat, col, lit
from pyspark.sql.types import IntegerType

# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [109]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [110]:
stack_overflow_data = 'Train_onetag_small.json'

In [111]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Body Length Feature

In [112]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [113]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [114]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [115]:
df.show(1)

+--------------------+---+--------------------+--------------------+------+--------------------+----------+
|                Body| Id|                Tags|               Title|oneTag|               words|BodyLength|
+--------------------+---+--------------------+--------------------+------+--------------------+----------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|   php|[p, i, d, like, t...|        83|
+--------------------+---+--------------------+--------------------+------+--------------------+----------+
only showing top 1 row



In [116]:
df.columns

['Body', 'Id', 'Tags', 'Title', 'oneTag', 'words', 'BodyLength']

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [117]:
# TODO: write your code to answer question 1
df.select('BodyLength').where(df.Id==1112).show()

+----------+
|BodyLength|
+----------+
|        63|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [118]:
# TODO: write your code to answer question 2
df =df.withColumn("Title_Body",concat(col("Title"),lit(','), col("Body")))

In [119]:
df.show(1)

+--------------------+---+--------------------+--------------------+------+--------------------+----------+--------------------+
|                Body| Id|                Tags|               Title|oneTag|               words|BodyLength|          Title_Body|
+--------------------+---+--------------------+--------------------+------+--------------------+----------+--------------------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|   php|[p, i, d, like, t...|        83|How to check if a...|
+--------------------+---+--------------------+--------------------+------+--------------------+----------+--------------------+
only showing top 1 row



In [120]:
regexTokenizer = RegexTokenizer(inputCol="Title_Body", outputCol="tb_token", pattern="\\W")
df = regexTokenizer.transform(df)
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [121]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("tb_len", body_length(df.tb_token))

In [122]:
df.head(1)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

In [123]:
df.select('tb_len').where(df.Id==5123).show()

+------+
|tb_len|
+------+
|   135|
+------+



# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [124]:
# TODO: write your code to create this vector
number_of_paragraphs = udf(lambda x: len(re.findall("</p>", x)), IntegerType())
number_of_links = udf(lambda x: len(re.findall("</a>", x)), IntegerType())

In [125]:
df = df.withColumn("NumParagraphs", number_of_paragraphs(df.Title_Body))
df = df.withColumn("NumLinks", number_of_links(df.Title_Body))

In [126]:
assembler = VectorAssembler(inputCols=["tb_len", "NumParagraphs", "NumLinks"], outputCol="NumFeatures")
df = assembler.transform(df)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [127]:
# TODO: write your code to answer question 3
scaler1 = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures1")
df_scaler1 = scaler1.transform(df)

# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [131]:
# TODO: write your code to answer question 4
scaler2 = StandardScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures2", withStd=True)
scalerModel = scaler2.fit(df)
df_scaler2 = scaler2.transform(df)

# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [134]:
# TODO: write your code to answer question 5
scaler3 = MinMaxScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures3")
scalerModel = scaler3.fit(df)
df_scaler3 = scaler2.transform(df)
